# TF

In [24]:
import pandas as pd
import numpy as np

from pandas.errors import EmptyDataError

# 정규식 활용 -> 특수문자 제거
import re

# 한글 형태소 분석기
from konlpy.tag import Okt
okt = Okt()

# 단어 빈도수 세기
from collections import Counter

In [25]:
date_str = '20250819'

In [ ]:
def safe_read_csv(path):
    try:
        return pd.read_csv(path)
    except EmptyDataError:
        print(f"⚠️ {path} 는 비어있음 → 스킵")
        return pd.DataFrame()   # 빈 DF 반환
    except FileNotFoundError:
        print(f"❌ {path} 파일 없음 → 스킵")
        return pd.DataFrame()

# 파일 불러오기
raw_data_i = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_i_{date_str}.csv")
raw_data_g = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_g_{date_str}.csv")
raw_data_gen_1 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_00000-19999.csv")
raw_data_gen_2 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_20000-39999.csv")
raw_data_gen_3 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_40000-59999.csv")
raw_data_gen_4 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_60000-79999.csv")
raw_data_gen_5 = safe_read_csv(f"/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_{date_str}_80000-99999.csv")

# 하나의 파일로 유니온 (비어있으면 자동 무시)
raw_data = pd.concat([
    raw_data_gen_1, raw_data_gen_2, raw_data_gen_3,
    raw_data_gen_4, raw_data_gen_5,
    raw_data_i, raw_data_g
], ignore_index=True)

print("총 데이터 크기:", raw_data.shape)

# 완전일치 중복기사 제거
df = raw_data.drop_duplicates(subset=['title', 'text'], keep='first').copy()

# title에 [속보] 또는 [포토] 포함 시 text를 NaN으로 변경
df.loc[df['title'].str.contains(r'\[속보\]|\[포토\]', regex=True), 'text'] = np.nan

# null셀을 빈칸으로 만들기
df['title'] = df['title'].fillna('')
df['text'] = df['text'].fillna('')

# 특수문자 등 전처리
df['text_clean'] = df['text'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df['title_clean'] = df['title'].apply(lambda t: re.sub(r'[^가-힣A-Za-z&\$₩]', ' ', t))
df.head(3)


###########################


# 불용어 불러오기 (줄바꿈 기준 분리)
with open("/Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/stopwords_kor.txt", "r", encoding="utf-8") as f:
    stopwords = f.read()

# 불용어 제거 함수
def remove_stopwords(text_clean):
    if not isinstance(text_clean, str):
        return ''
    return ' '.join([word for word in text_clean.split() if word not in stopwords])

# 조사 제거 함수
def remove_josa(text):
    if not isinstance(text, str):
        return ''
    return ' '.join([word for word, pos in okt.pos(text) if pos != 'Josa'])

# 조사 제거 → 불용어 제거 순차 적용
df['title_clean'] = df['title_clean'].apply(remove_josa)  # 조사 제거
df['title_clean'] = df['title_clean'].apply(remove_stopwords)  # 불용어 제거

df['text_clean'] = df['text_clean'].apply(remove_josa)  # 조사 제거
df['text_clean'] = df['text_clean'].apply(remove_stopwords)  # 불용어 제거



# 1. 전체 단어 모으기
all_words = []
for text in df['title_clean']:
    if isinstance(text, str):
        all_words.extend(text.split())  # 공백 기준 토큰 분리

# 2. 빈도 계산
word_counts = Counter(all_words)

# 3. DataFrame 변환
freq_df = pd.DataFrame(word_counts.items(), columns=['word', 'count'])
freq_df = freq_df.sort_values(by='count', ascending=False).reset_index(drop=True)

freq_df # 상위 20개 단어

⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250819_00000-19999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250819_20000-39999.csv 는 비어있음 → 스킵
⚠️ /Users/leesangwon/Documents/ThemeStock_file/Hankyung_news/hankyung_gen_20250819_40000-59999.csv 는 비어있음 → 스킵
총 데이터 크기: (619, 4)


,word,count
0,속보,28
1,K,16
2,젤,16
3,렌스,15
4,삼성,14
...,...,...
2037,야외,1
2038,DDP,1
2039,구름,1
2040,분홍,1


In [14]:
freq_df.to_csv(f"/Users/leesangwon/Documents/ThemeStock_file/title_TF_{date_str}.csv", index=False)